![alt text](https://www.unsw.edu.au/sites/default/files/UNSW_0.png)

# How to compute image features with OpenCV


---

In [1]:
#install opencv
!pip install opencv-contrib-python==4.1.2.30 


In [1]:
import cv2 
import glob
import pandas as pd
import numpy as np
%matplotlib inline
import pywt



# The functions

In [61]:
def Depth_of_feild (image):
    
    # Define the window size
    windowsize_r = round(image.shape[0]/4)
    windowsize_c = round(image.shape[1]/4)

    # devide the image into blocks
    window = np.zeros(shape=(windowsize_r,windowsize_c,3,16)).astype('uint8')
    k = 0
    for r in range(0,image.shape[0] - windowsize_r, windowsize_r):
        for c in range(0,image.shape[1] - windowsize_c, windowsize_c):
            window[:,:,:,k] = image[r:r+windowsize_r,c:c+windowsize_c]
            k = k +1
            #display objects

    #center blocks
    center = [5,6,9,10]

    #calaculate the wavelet coeff. for center blocks
    sum_center_blocks = np.zeros(shape = [3])
    for i in center:
        block = window[:,:,:,i]
        Sums = calc_wavelet (block)
        sum_center_blocks = sum_center_blocks + Sums

    #calaculate the wavelet coeff. for whole image 
    Sum_whole_image = calc_wavelet (image)

    #return  the depth of feild for each channel (H,S, and V)
    return abs(sum_center_blocks/Sum_whole_image)


In [62]:
def calc_wavelet (block):
    hsv = cv2.cvtColor(block, cv2.COLOR_BGR2HSV)
    sums =[]
    for i in range(3):
        Channel = hsv[:,:,i]
        coffes_H=pywt.dwt2(Channel,'db1')
        ca,(ch,cv,cd)= coffes_H
        sums.append (np.sum(cd))
    return sums

In [63]:
def Difference(opened_file):
    
    #resize image to reduce processing time 
    opened_file = cv2.resize(opened_file, (0, 0), fx=0.5, fy=0.5)
    
    # initialize OpenCV's static fine grained saliency detector and
    # compute the saliency map

    #image = cv2.imread('C:\\Users\\oalq0001\\Downloads\\saliency-detection\\images\\players.jpg')
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliencyMap) = saliency.computeSaliency(opened_file)
    # chamge the scale into 0-255
    new_arr = ((saliencyMap - saliencyMap.min()) * (1/(saliencyMap.max() - saliencyMap.min()) * 255))

    # if we would like a *binary* map that we could process for contours,
    # compute convex hull's, extract bounding boxes, etc., we can
    # additionally threshold the saliency map
    threshMap = cv2.threshold(new_arr.astype('uint8'), 0, 255, cv2.THRESH_OTSU)[1]

    #display objects
    #cv2.imshow("Image", threshMap)
    #cv2.waitKey(0)
    
    # calcualte Area difference
    objects = np.sum(threshMap/255)
    background = (threshMap.shape[0]*threshMap.shape[1]) - objects
    Area_difference = (objects - background)/(threshMap.shape[0]*threshMap.shape[1])
    
    # calculate Colordifference
    Objects_image = cv2.bitwise_and(opened_file,opened_file,mask = (threshMap/255).astype('uint8'))
    Background_image = cv2.bitwise_and(opened_file,opened_file,mask = ~(threshMap).astype('uint8'))
    
    R_Obj = np.sum(Objects_image[:,:,0])/objects
    G_Obj = np.sum(Objects_image[:,:,1])/objects
    B_Obj = np.sum(Objects_image[:,:,2])/objects
    
    R_back = np.sum(Background_image[:,:,0])/background
    G_back = np.sum(Background_image[:,:,1])/background
    B_back = np.sum(Background_image[:,:,2])/background
    
    Color_difference = ((R_Obj - R_back)**2 + (G_Obj - G_back)**2 + (B_Obj - B_back)**2)**.5
    
    #calculate Texture difference
    edges_objects = cv2.Canny(Objects_image, 50, 150, apertureSize=3)
    edges_background = cv2.Canny(Background_image, 50, 150, apertureSize=3)

    edges_objects_density = np.sum(edges_objects/255)/objects
    edges_background_density = np.sum(edges_background/255)/background

    Texture_difference = abs(edges_background_density - edges_objects_density)



    return (Area_difference,Color_difference,Texture_difference)

In [74]:
def brightness(opened_file):
  Bright = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY)/255.0

  return round(Bright.mean(), 2)

In [65]:
def saturation(opened_file):
  hsv = cv2.cvtColor(opened_file, cv2.COLOR_BGR2HSV)

  # saturation is the s channel
  s = hsv[:, :, 1]

  return round(s.mean(), 2)

In [66]:
def contrast_of_brightness(opened_file):
  gray = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY)/255.0

  return round(gray.std(), 2)

In [67]:

def image_clarity(opened_file):
  gray = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY) / 255.0
  bright = gray >= .7

  return round(bright.sum() / bright.size, 2)

In [68]:
def warm_hue(opened_file):
  hsv = cv2.cvtColor(opened_file, cv2.COLOR_BGR2HSV)

  # hue is the h channel
  h = hsv[:, :, 0]
  warm = ~ (h > 30) & (h < 210)

  return round(warm.sum() / warm.size, 2)

In [69]:
def image_colorfulness(opened_file):
  # split the image into its respective RGB components
  (B, G, R) = cv2.split(opened_file.astype("float"))

  # compute rg = R - G
  rg = np.absolute(R - G)

  # compute yb = 0.5 * (R + G) - B
  yb = np.absolute(0.5 * (R + G) - B)

  # compute the mean and standard deviation of both `rg` and `yb`
  (rbMean, rbStd) = (np.mean(rg), np.std(rg))
  (ybMean, ybStd) = (np.mean(yb), np.std(yb))

  # combine the mean and standard deviations
  std_root = np.sqrt((rbStd ** 2) + (ybStd ** 2))
  mean_root = np.sqrt((rbMean ** 2) + (ybMean ** 2))

  # derive the "colorfulness" metric and return it
  return round(std_root + (0.3 * mean_root))

In [76]:
def horizontal_color_balance(opened_file):
  mid = int(opened_file.shape[1] / 2)
  left_half = np.array(opened_file[:, 0:mid, ], dtype='int')
  right_half = np.flip(np.array(opened_file[:, mid:2 * mid, ],
                                      dtype='int'), axis=1)
  dif_square = np.square(left_half - right_half)
  euclidean = np.sqrt(dif_square.sum(axis=2))

  return round(-euclidean.mean(), 2)

In [77]:
def vertical_color_balance(opened_file):
  mid = int(opened_file.shape[0] / 2)
  upper_half = np.array(opened_file[0:mid,: , ], dtype='int')
  lower_half = np.flip(np.array(opened_file[mid:2 * mid,: , ],
                                      dtype='int'), axis=1)
  dif_square = np.square(upper_half - lower_half)
  euclidean = np.sqrt(dif_square.sum(axis=2))

  return round(-euclidean.mean(), 2)

# The main function

In [105]:
def collect_features (filename):
    image = cv2.imread(filename)
    id = filename.split('\\')[-1].split('.')[0]
    Brightness = brightness (image)
    Saturation = saturation (image)
    Contrast = contrast_of_brightness (image)
    Clarity = image_clarity (image)
    Warm_hue = warm_hue (image)
    Colorfulness = image_colorfulness (image)
    Vertical_color_balance =   vertical_color_balance(image)
    Horizontal_color_balance = horizontal_color_balance(image)
    (Size_difference,Color_difference, Texture_difference) = Difference(image)
    [Depth_of_field_hue, Depth_of_field_saturation, Depth_of_field_saturation_value] = Depth_of_feild (image)
    result  = {'Id':id,'Brightness':Brightness,'Saturation':Saturation,'Contrast':Contrast,
               'Clarity':Clarity,'Warm Hue':Warm_hue, 'Colorfulness': Colorfulness, 
               'Vertical color balance':Vertical_color_balance,
               'Horizontal color balance': Horizontal_color_balance,
               'Size_difference':Size_difference,'Color difference':Color_difference,
               'Texture difference':Texture_difference, 'Depth of field_hue':Depth_of_field_hue,
               'Depth of field_saturation':Depth_of_field_saturation,
               'Depth of field_saturation_value':Depth_of_field_saturation_value}
    return result
        

# select the location of the images

In [106]:
# make sure to put the right path. Here ()test images) is in the root of my google drive
list_of_files = glob.glob('C:\\Users\\oalq0001\\Downloads\\saliency-detection\\images\\*.*')

In [139]:
# create an empty list
features = []
#loop over list_of_files
for i,filename in enumerate (list_of_files):
      
    print('processing image no.:',i)
    
    # send the image nmae to collect_features function append theh results to list
    collected = collect_features (filename)
    features.append(collected)


processing image no.: 0
processing image no.: 1
processing image no.: 2
processing image no.: 3


In [152]:
# save the results in a DataFrame
data = pd.DataFrame(features)


In [153]:
data.head()

,Brightness,Clarity,Color difference,Colorfulness,Contrast,Depth of field_hue,Depth of field_saturation,Depth of field_saturation_value,Horizontal color balance,Id,Saturation,Size_difference,Texture difference,Vertical color balance,Warm Hue
0,0.36,0.09,39.131044,46.0,0.20,0.084738,0.297619,0.086226,-78.87,barcelona,109.31,-0.334400,0.149683,-124.41,0.16
1,0.30,0.01,56.417324,60.0,0.13,0.335165,1.105145,0.378788,-32.80,boat,214.23,-0.861304,0.341556,-87.34,0.00
2,0.54,0.02,38.956438,35.0,0.08,5.646341,0.615132,0.366126,-34.32,neymar,132.37,-0.938410,0.289957,-35.78,0.03
3,0.36,0.01,47.462881,35.0,0.11,0.094041,0.297535,0.272801,-49.89,players,105.25,-0.791172,0.349534,-55.26,0.35


## Normalize the data to become in the range (0-1)

In [154]:
ids = data.pop('Id')
normalized = (data-data.min())/(data.max()-data.min())
normalized.index = ids
normalized.head()

,Brightness,Clarity,Color difference,Colorfulness,Contrast,Depth of field_hue,Depth of field_saturation,Depth of field_saturation_value,Horizontal color balance,Saturation,Size_difference,Texture difference,Vertical color balance,Warm Hue
Id,,,,,,,,,,,,,,
barcelona,0.25,1.000,0.010000,0.44,1.000000,0.000000,0.000104,0.000000,0.000000,0.037255,1.000000,0.000000,0.000000,0.457143
boat,0.00,0.000,1.000000,1.00,0.416667,0.045028,1.000000,1.000000,1.000000,1.000000,0.127656,0.960083,0.418256,0.000000
neymar,1.00,0.125,0.000000,0.00,0.000000,1.000000,0.393255,0.956720,0.967007,0.248853,0.000000,0.701892,1.000000,0.085714
players,0.25,0.000,0.487171,0.00,0.250000,0.001673,0.000000,0.637729,0.629043,0.000000,0.243766,1.000000,0.780210,1.000000


# save the output

In [156]:
normalized.to_csv('output.csv')